In [0]:
import sys
import os

sys.path.append(os.path.abspath('..'))

from src.utilities import *

In [0]:
TABLE_SILVER_EXP = "silver_dev.sctr_emision.expuestos_silver"
TABLE_SILVER_CONT = "silver_dev.sctr_emision.contratantes_silver"

TABLE_GOLD_BI = "gold_dev.sctr_emision.consolidado_gold"

open_log("Gold")

# spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")
logger.info(f"⚪ Iniciando proceso Gold en Databricks. Clúster: {spark.conf.get('spark.databricks.clusterUsageTags.clusterId')}")

In [0]:
def process_gold_consolidation() -> DataFrame:
    try:
        logger.info(f"   🔍 Leyendo Silver Expuestos...")
        df_exp = spark.read.table(TABLE_SILVER_EXP)

        total_rows = df_exp.count()
        if total_rows == 0: 
            logger.warning(f"   ⚠️ La tabla Silver Expuestos no contiene registros.")
            # raise Exception(f"La tabla Silver Expuestos no contiene registros.")
            return None

        logger.info(f"   🔍 Leyendo Silver Contratantes...")
        df_cont = spark.read.table(TABLE_SILVER_CONT)

        total_rows = df_cont.count()
        if total_rows == 0: 
            logger.warning(f"   ⚠️ La tabla Silver Contratantes no contiene registros.")
            return None

        logger.info(f"   🔄 Transformando Consolidado...")

        df_exp = df_exp.alias("A")
        df_cont = df_cont.alias("B")
        
        join_cond = (
            (F.col("A.POLIZA") == F.col("B.POLIZA")) 
            # &
            # (F.col("A.YEAR_MOV") == F.col("B.YEAR_MOV")) 
            # &
            # (F.col("A.MONTH_MOV") == F.col("B.MONTH_MOV"))
        )
        
        df_joined = df_exp.join(df_cont, join_cond, "left")
        
        cols_select = [
            F.col("A.POLIZA"),
            F.col("A.F_INI_VIGEN_POLIZA"),
            F.col("A.F_FIN_VIGEN_POLIZA"),
            F.col("A.F_INI_COBERT"),
            F.col("A.F_FIN_COBERT"),
            F.col("B.NUM_DOC_CONT"),
            F.col("B.CONTRATANTE"),
            F.col("A.TIPO_DOC"),
            F.col("A.NUM_DOC"),
            F.col("A.ULT_DIGI_DOC"),
            F.col("A.EXPUESTO")
        ]
        
        df_final = df_joined.select(cols_select).distinct()

        return df_final
    except Exception as e:
        logger.error(f"   ❌ Error en Consolidado Gold. {e}")
        return None

In [0]:
STATUS = False

try:
    df_gold = process_gold_consolidation()
    
    STATUS = True if df_gold is not None else False
    if STATUS:
        total_rows = df_gold.count()
        logger.info(f"   📊 Total Registros Guardados: {total_rows:,.0f}")
        STATUS = save_to_table_delta(df_gold, TABLE_GOLD_BI, "overwrite", "false")

    if validate_table_delta(TABLE_GOLD_BI):
        logger.info("   🧹 Optimizando tabla Gold Consolidado BI...")
        spark.sql(f"OPTIMIZE {TABLE_GOLD_BI} ZORDER BY (ULT_DIGI_DOC, NUM_DOC)")
        STATUS = True

    if STATUS:
        logger.success("🏁 Ejecución Completa: Proceso Gold Finalizado con éxito.")
    else:
        logger.error("🏁 Ejecución Incompleta: Proceso Gold Finalizado con Error.")
except Exception as e:
    logger.error(f"❌ Error crítico en proceso Gold. {e}")
finally:
    HORA_FINAL = datetime.now()
    difference_time = HORA_FINAL-HORA_INICIAL
    total_seconds = int(difference_time.total_seconds())
    difference_formated = "{} minuto(s), {} segundo(s)".format((total_seconds // 60), total_seconds % 60)
    logger.info(f"Tiempo de proceso: {difference_formated}")
    close_log()

07/01/2026 11:11:57 | AVISO   |    🔍 Leyendo Silver Expuestos...
07/01/2026 11:11:58 | AVISO   |    🔍 Leyendo Silver Contratantes...
07/01/2026 11:11:58 | AVISO   |    🔄 Transformando Consolidado...
07/01/2026 11:12:12 | AVISO   |    📊 Total Registros Guardados: 9,255,392
07/01/2026 11:12:30 | AVISO   |    ✅ La tabla Delta existe en gold_dev.sctr_emision.consolidado_gold.
07/01/2026 11:12:30 | AVISO   |    🧹 Optimizando tabla Gold Consolidado BI...
07/01/2026 11:12:44 | ÉXITO   | 🏁 Ejecución Completa: Proceso Gold Finalizado con éxito.
07/01/2026 11:12:44 | AVISO   | Tiempo de proceso: 1 minuto(s), 1 segundo(s)
07/01/2026 11:12:44 | AVISO   | 🏁 Finalizando log y forzando última sincronización...
✅ Log guardado exitosamente en: /Volumes/landing_dev/sctr_emision/inputs_volumen/Logs/ETL_Emision_SCTR_Gold_2026-01-07.log
🧹 Archivo temporal local eliminado.
